# Multiprocessing: limited direct execution
_COSC 208, Introduction to Computer Systems, 2023-04-12_

## Announcements
* DEI assignment 3 due Thursday @ 11pm
* Project 4 due Thursday, April 20

## Outline
* Warm-up: Optimizing loops for locality
* Operating Systems (OS)
* Accessing hardware

## Warm-up: Optimizing loops for locality (Q5-Q8 from previous class)

* Q1: _Modify the following function to better leverage spatial locality_

In [ ]:
void hundreds() {
    int *nums = malloc(sizeof(int) * 1000);
    for (int i = 0; i < 100; i++) {
        for (int j = 0; j < 1000; j+= 100) {
            nums[i+j] = i;
        }
    }
}

In [ ]:
// Perform loop interchange
void hundreds_optimized() {
    int *nums = malloc(sizeof(int) * 1000);
    for (int j = 0; j < 1000; j+= 100) {
        for (int i = 0; i < 100; i++) {
            nums[i+j] = i;
        }
    }
}

<p style="height:4em;"></p>

* Q2: _Modify the following function to improve temporal locality_

In [ ]:
int odds(int *nums, int length) {
    for (int i = 0; i < length; i++) {
        nums[i] = nums[i] % 2;
    }
    int count = 0;
    for (int j = 0; j < length; j++) {
        count += nums[j];
    }
    return count;
}

In [ ]:
// Perform loop fusion
int odds_optimized(int *nums, int length) {
    int count = 0;
    for (int i = 0; i < length; i++) {
        nums[i] = nums[i] % 2;
        count += nums[i];
    }
    return count;
}

<p style="height:4em;"></p>

* Q3: _Modify the following function to better leverage spatial locality_

In [ ]:
void multiplication(int grid[][], int rows, int cols) {
    for (int c = 0; c < cols; c++) {
        for (int r = 0; r < rows; r++) {
            grid[r][c] = c * r;
        }
    }
}

In [ ]:
// Perform loop interchange
void multiplication_optimized(int grid[][], int rows, int cols) {
    for (int r = 0; r < rows; r++) {
        for (int c = 0; c < cols; c++) {
            grid[r][c] = c * r;
        }
    }
}

<div style="page-break-after:always;"></div>

* Q4: _Modify the following function to improve temporal locality_

In [ ]:
long stdev(int *nums, int length) {
    long sum = 0;
    for (int i = 0; i < length; i++) {
        sum += nums[i];
    }
    int mean = sum / length;
    sum = 0;
    for (int j = 0; j < length; j++) {
        int diff = nums[j] - mean;
        sum += diff * diff:
    }
    mean = sum / length;
    return sqrt(mean);
}

    It's not possible to optimize this code more:
    no loops to interchange and loops cannot be fused.

<p style="height:12em;"></p>

## Operating systems (OS) overview

* Purpose of an OS
    * Make computer hardware easy to use—e.g., an OS knows how to load an application's executable code from persistent storage (e.g., solid state drive (SSD)) into main memory, initialize the process's address space (code, heap, stack), and make the CPU execute the application's instructions
    * Support multiprocessing—i.e., running multiple applications concurrently
        * Concurrently == switch between multiple tasks during a window of time—e.g., alternate between cooking and setting the table
        * Simultaneously == complete two tasks at the same time—e.g., listen to a podcast while cooking
    * Allocate and manage hardware resources—e.g., decide when/which applications can use the CPU, decide when/which memory applications can use, prevent applications from stealing/accessing another application's CPU time or memory
    * Many OSes also provide a user interface (UI)
* How does the OS fulfill its duties?
    * Mechanisms — fundamental approaches for managing/providing access to hardware resources
        * E.g., system calls, process abstraction
    * Policy — specific ways of employing an approach; different policies make different trade-offs (in terms of efficiency, performance, etc.)
        * E.g., process scheduler

## Accessing hardware

* OS is responsible for allocating/managing the hardware
    * ⇒ applications should **not have unfettered access to hardware**
* _How should applications access the hardware?_
    * Ask the OS for access to the hardware
        * How do we ensure the OS does not "lose control" of the hardware? 
    * Asks the OS to perform an action on the application's behalf
        * How do we ensure this doesn't substantially degrade performance?
* Example: execute an instruction on the CPU
    * Asking the OS to do this on behalf of an application is impractical—OS would need to execute multiple assembly instructions for each assembly instruction the application wants to execute
    * Alternative: allow the application to execute its own instructions on the CPU
        * This is risky—an application may execute an instruction that controls the hardware, e.g., `hlt` (halt) instruction pauses the CPU
    * Alternative: allow the application to execute "safe" instructions on its own on the CPU
* Example: accessing the solid state drive (SSD)
    * Allowing the application to access the SSD directly
        * This is risky—an application may read/write data it should not be able to access
    * Alternative: asking the OS to access the SSD on the application's behalf
        * Latency of accessing SSD (~1 million CPU cycles) far outweighs the extra instructions required for OS to perform the access on the application's behalf
* Mechanisms
    * Limited Direct Execution (LDE)
    * System calls

## Extra practice

* Q5: _Modify the `prediction` function to improve locality_

In [ ]:
char prediction(int *votes, char *projection) {
    int democrat = 0;
    int republican = 0;
    int length = strlen(projection);
    for (int d = 0; d < length; d++) {
        if (projection[d] == 'D') {
            democrat += votes[d];
        }
    }
    for (int r = 0; r < length; r++) {
        if (projection[r] == 'R') {
            republican += votes[r];
        }
    }
    if (democrat > republican) {
        return 'D';
    }
    else {
        return 'R';
    }
}

In [ ]:
// Perform loop fusion
char prediction(int *votes, char *projection) {
    int democrat = 0;
    int republican = 0;
    int length = strlen(projection);
    for (int v = 0; v < length; v++) {
        if (projection[v] == 'D') {
            democrat += votes[v];
        }
        if (projection[v] == 'R') {
            republican += votes[v];
        }
    }
    if (democrat > republican) {
        return 'D';
    }
    else {
        return 'R';
    }
}

<p style="height:4em;"></p>

* Q6: _Modify the `rowSum` function to improve locality_

In [ ]:
int *rowSum(int grid[][], int rows, int cols) {
    int *sums = malloc(sizeof(int) * rows);
    for (int i = 0; i < rows; i++) {
        sum[i] = 0;
    }
    for (int r = 0; r < rows; r++) {
        for (int c = 0; c < cols; c++) {
            sums[r] += grid[r][c];
        }
    }
    return sums;
}

In [ ]:
// Perform loop fusion
int *rowSum(int grid[][], int rows, int cols) {
    int *sums = malloc(sizeof(int) * rows);
    for (int r = 0; r < rows; r++) {
        sum[r] = 0;
        for (int c = 0; c < cols; c++) {
            sums[r] += grid[r][c];
        }
    }
    return sums;
}